# Calling Google Speech-to-Text (STT) Recognizer

In [ ]:
import argparse
import io
from google.cloud import speech_v1p1beta1 as speech

def transcribe_file_with_word_time_offsets(speech_file):
    
    """Transcribe the given audio file synchronously and output the word time
    offsets."""
    from google.cloud import speech

    client = speech.SpeechClient()

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000, #For best performance
        language_code="en-US",   #Change accroding to the examiner voice accent
        enable_word_time_offsets=True,
        #model = 'phone_call',
        #use_enhanced=True,
    )
    global response
    response = client.recognize(config=config, audio=audio)
    
    for result in response.results:
        #alternative = []
        global alternative
        
        alternative = result.alternatives[0]

        for word_info in alternative.words:
            
            
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time

# Graphical User Interface (GUI) Development

In [ ]:
from tkinter import *
from pygame import mixer
import tkinter as tk
from scipy.io import wavfile
import IPython.display as ipd
from tkinter import filedialog
from tkinter import simpledialog
import math
import threading
import time
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import time
import matplotlib.pyplot as plot
from threading import *
import os
from tkinter import ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)
import simpleaudio as sa
from PIL import Image, ImageTk
import malaya_speech
from datetime import timedelta
from tkinter import messagebox
from tkinter.messagebox import askyesno

window=tk.Tk()
window.title('Graphical User Interface for Automatic Heart Voice Detection')
window.geometry("400x250+10+20")


global i, h
i = 0 
h = 0


def threading():
    
    # Call work function
    t1=Thread(target=process_and_check)
    t1.start()

############### Selecting the Input File ###############

def select_input_file():
    
    global filename_base
    global data, samplerate, L, y, x, z, pp
    
    filename_base = filedialog.askopenfilenames(initialdir=r'C:\Users')
    samplerate, data = wavfile.read(filename_base[0])
    data = 20*data       #Amplifying data for better recognizer accuracy
    x = math.ceil((data.shape[0])/samplerate) #Length of the data in seconds
    L = 59               #Duration of the data to be processed in seconds
    y = math.ceil(x/L)
    pp = round(100/y, 2) #Progress bar percentage
    print(data.shape)
    print(samplerate)
    print(y)
    print(x)
    ##= int((100/y)*100)/100
    import numpy as np
    x = np.zeros((y,2))
    z = np.empty(y, dtype='object')
    #user_input()

############### Processing the Audio Chunks ###############
    
def process_and_check():
    
    for i in range(y):
        global header_folder, heart_voice_folder, main_folder
        global time_to_int_starting, time_to_int_ending
        global selected_data
        
        Prog_bar['value'] += pp
        xxx = round(Prog_bar['value'],2)
        Prog_bar['value'] = xxx
        if Prog_bar['value'] > 97:
            Prog_bar['value'] = 100
        window.update_idletasks()
        my_progress.config(text = Prog_bar['value'])
    
        selected_data = data[i*L*samplerate:(i+1)*L*samplerate]

        main_folder = os.path.split(filename_base[0])[0]
        header_folder = os.path.join(main_folder, 'Header')
        heart_voice_folder = os.path.join(main_folder, 'Heart')

    
        global x, z
        wavfile.write(os.path.join(main_folder,'test'+str(i)+'.wav'), samplerate, selected_data)
        speech_file = os.path.join(main_folder,'test'+str(i)+'.wav')
        transcribe_file_with_word_time_offsets(speech_file)
        os.remove(os.path.join(main_folder,'test'+str(i)+'.wav')) 
    
        start_of_speaker = alternative.words[0].start_time
        end_of_speaker   = alternative.words[-1].end_time

        b = (i)*timedelta(0, L) 

        new_a = start_of_speaker + b 
        new_a2 = end_of_speaker + b
        print(new_a)
        print(new_a2)
        new_a = int(new_a.total_seconds()*samplerate)
        new_a2 = int(new_a2.total_seconds()*samplerate)
        x[i] = [new_a, new_a2]
        z[i] = alternative.transcript
        print(z[i])
        if z[i] == z[i-1] or z[i] == z[i-2]:
            z[i] = ' '
        print(z[i])
        print(i)
        time_to_int_starting = int(start_of_speaker.total_seconds()*samplerate)
        time_to_int_ending = int(end_of_speaker.total_seconds()*samplerate)
    program_end_message()

    
############### End of Processing: Notifying the End-user ###############

def program_end_message():
    
    
    messagebox.showinfo("Program",
                           "Program has been finished! Now select Audio chunking and selection!")
       
def audio_chunking_and_selection():
    
    
    global aa, bb
    human_voice = {}
    aa = {}
    bb = {}
    j = 0
    for i in range(y):
        if z[i] != ' ' and (x[i][1] - x[i][0]) > 3*samplerate:
            
            aa[j] = int(x[i][0])
            bb[j] = int(x[i][1])
            print(j)
            j += 1

def index_check():
    
    messagebox.showinfo("Program",
                        "You may want to close the GUI!")
    
    time.sleep(3)
    window.destroy()
    

############### Plotting the Spectrograms ###############
        
def plot_spectrogram():
    
    
#     if h!= len(aa):
    
    global h
    win = tk.Toplevel()
    win.geometry("750x500")
    
    def norm_mel(y, sr):
        mel = librosa.feature.melspectrogram(y, sr = sr)
        mel_dB = librosa.power_to_db(mel, ref=np.max)
        return mel_dB #np.log10(np.maximum(mel, 1e-10)).T
    
    def plot1(y, sr):    
        mel_dB = norm_mel(y, sr)
        fig, axs = plt.subplots(1, figsize=(8, 3))
        img = librosa.display.specshow(mel_dB, x_axis='time',
                                       y_axis='mel', sr=sr, ax=axs)
        plt.savefig(os.path.join(main_folder, 'test'+str(i)+'.jpg'))
        plt.show()        
        
    canvas= Canvas(win, width= 600, height= 500)
    canvas.pack()
    # First Image
    if h != len(aa)-1:
        
        heart_data_ = malaya_speech.astype.int_to_float(data[aa[h]:bb[h]])
        plot1(heart_data_, samplerate)    
        img = ImageTk.PhotoImage(Image.open(os.path.join(main_folder, 'test'+str(i)+'.jpg')))
        canvas.create_image(0,0,anchor=NW,image=img)
        
        # Second Image
        heart_data_ = malaya_speech.astype.int_to_float(data[bb[h]:aa[h+1]])
        plot1(heart_data_, samplerate)
        img2 = ImageTk.PhotoImage(Image.open(os.path.join(main_folder, 'test'+str(i)+'.jpg')))
        canvas.create_image(0,220,anchor=NW,image=img2)
    
    elif h == len(aa)-1:
        
        heart_data_ = malaya_speech.astype.int_to_float(data[aa[h]:bb[h]])
        plot1(heart_data_, samplerate)    
        img = ImageTk.PhotoImage(Image.open(os.path.join(main_folder, 'test'+str(i)+'.jpg')))
        canvas.create_image(0,0,anchor=NW,image=img)
        
        # Second image
        heart_data_ = malaya_speech.astype.int_to_float(data[bb[h]:])
        plot1(heart_data_, samplerate)
        img2 = ImageTk.PhotoImage(Image.open(os.path.join(main_folder, 'test'+str(i)+'.jpg')))
        canvas.create_image(0,220,anchor=NW,image=img2)
    
    play_music()
    
    win.mainloop()

############### Playing the Audio Files for End-user ###############    
    
def play_music():
    

    global h
    #mixer.init()
    global USER_INP
    
    if h != len(aa)-1:
        play_obj  = sa.play_buffer(data[aa[h]:bb[h]], 1, 2, samplerate)
        #start_of_speaker = alternative.words[0].start_time
        #end_of_speaker   = alternative.words[-1].end_time
        start_of_speaker = bb[h]
        end_of_speaker   = aa[h]
        duration_in_seconds = int((start_of_speaker - end_of_speaker)/samplerate)
        #time.sleep(end_of_speaker.total_seconds() - start_of_speaker.total_seconds() + 2)
        time.sleep(duration_in_seconds + 2)
        play_obj2 = sa.play_buffer(data[bb[h]:aa[h+1]], 1, 2, samplerate)
    
    elif h == len(aa)-1:
        play_obj  = sa.play_buffer(data[aa[h]:bb[h]], 1, 2, samplerate)
        #start_of_speaker = alternative.words[0].start_time
        #end_of_speaker   = alternative.words[-1].end_time
        start_of_speaker = bb[h]
        end_of_speaker   = aa[h]
        duration_in_seconds = int((start_of_speaker - end_of_speaker)/samplerate)
        #time.sleep(end_of_speaker.total_seconds() - start_of_speaker.total_seconds() + 2)
        time.sleep(duration_in_seconds + 2)
        play_obj2 = sa.play_buffer(data[bb[h]:], 1, 2, samplerate)
    
    USER_INP = simpledialog.askstring(title="", prompt="Do you want to save it?")


    save_files()
    
          
def save_files():
    
    global h
    h += 1

    if h-1 != len(aa):
        if USER_INP == 'yes':
            if h-1 != len(aa)-1:
                wavfile.write(os.path.join(header_folder,'Human_voice'+str(h-1)+'.wav'), samplerate, data[aa[h-1]:bb[h-1]])
                wavfile.write(os.path.join(heart_voice_folder,'Heart_sound'+str(h-1)+'.wav'), samplerate, data[bb[h-1]:aa[h]])
            elif h-1 == len(aa)-1:
                wavfile.write(os.path.join(header_folder,'Human_voice'+str(h-1)+'.wav'), samplerate, data[aa[h-1]:bb[h-1]])
                wavfile.write(os.path.join(heart_voice_folder,'Heart_sound'+str(h-1)+'.wav'), samplerate, data[bb[h-1]:])
        
         

        plot_spectrogram()
    elif h == len(aa):
        
        index_check()

############### Creating Buttons for GUI ###############
            
topframe = Frame(window)
topframe.pack( side = TOP )

bottomframe1 = Frame(window)
bottomframe1.pack( side = TOP )

bottomframe2 = Frame(window)
bottomframe2.pack( side = TOP )

bottomframe = Frame(window)
#bottomframe.pack( side = TOP )

lastframe = Frame(window)
lastframe.pack( side = BOTTOM )
v = tk.IntVar()

tk.Label(bottomframe2, 
        text="""Choose a file to play:""",
        justify = tk.LEFT,
        padx = 20)

B_set = tk.Button(topframe, text="Select file", width = 10, height = 5,
               padx = 5, pady= 1, command = select_input_file)
B_set.pack(side = LEFT, padx = 5)

B0 = tk.Button(topframe, text="Start to process",
               command = threading,
               height = 5,
               width = 10,
               padx = 5, pady = 1)
B0.pack(side = RIGHT, padx = 5)

Prog_bar = ttk.Progressbar(bottomframe1, orient = HORIZONTAL, length = 200, mode = 'determinate')
Prog_bar.pack(pady = 5)

Percentage_sign = tk.Label(bottomframe1, text = "%")
Percentage_sign.pack(side = RIGHT, pady = 1)

my_progress = tk.Label(bottomframe1, text = "")
my_progress.pack(side = RIGHT, pady = 1)

my_progress_label = tk.Label(bottomframe1, text = "Progress:")
my_progress_label.pack(side= LEFT, pady = 1)


B1 = tk.Button(bottomframe2, 
               command = audio_chunking_and_selection,
               text="Audio chunking and selection",
               padx = 20)
B1.pack(pady = 1)
B2 = tk.Button(bottomframe2, 
               command = plot_spectrogram,
               text="Play and save",
               padx = 20)

B2.pack(pady = 1)

tk.Button(lastframe, 
        text="Exit",
          command = window.destroy,
          width = 10,
          height = 2,
        padx = 5).pack()
window.mainloop()